# Connectivity matrices

## Extraction and plotting

Here I read the atlas partitioned datasets and calculate the connectivity matrix for each subject

In [33]:
%matplotlib inline

import h5py
import os.path           as     op
import numpy             as     np
import matplotlib.pyplot as     plt
import seaborn           as     sns
from   functools         import partial
from   natsort           import natsorted
from   collections       import OrderedDict

from   boyle.storage            import (get_dataset_names, get_group_names, get_datasets, 
                                        save_variables_to_hdf5, extract_datasets)
from   luigi.similarity_measure import SimilarityMeasureFactory
from   luigi.selection          import TimeSeriesSelectorFactory
from   luigi.connectivity       import build_timeseries, transform_timeseries, calculate_connectivity

from   fabfile                  import get_subject_labels, get_filtered_subjects_ids_and_labels

In [34]:
timeseries_h5path     = '/Users/alexandre/Projects/bcc/cobre/cobre_partitioned_timeseries.hdf5'

TR                    = 2
build_timeseries      = partial(build_timeseries, sampling_interval=TR, pre_filter=None, normalize=None)

aalts_groupname       = '/pipe_wtemp_noglob_aal_3mm_func_timeseries'
aalconns_groupname    = '/pipe_wtemp_noglob_aal_3mm_connectivities'
subj_groups           = get_group_names(timeseries_h5path, aalts_groupname)
file_groups           = get_group_names(timeseries_h5path, '/')

subj_ids, subj_labels = get_filtered_subjects_ids_and_labels()

In [3]:
def get_subject_timeseries(h5file_path, subj_path, sampling_interval=TR):
    """Return the timeseries of one subject in a HDF5 file.
    
    Parameters
    ----------
    h5file_path: str
        Path to the hdf5 file with the subject timeseries.
    
    subj_path: str
        HDF5 internal path to the subject.

    sampling_interval: int or float
        Timeseries sampling interval in seconds.

    Returns
    -------
    timeseries: OrderedDict of nitime.timeseries.TimeSeries
        A dictionary with all the partition timeseries of the subject.
    """
    timeseries = OrderedDict()
    with h5py.File(h5file_path, mode='r') as timeseries_file:
        dspaths    = natsorted(get_dataset_names(timeseries_file, subj_path))
        for dspath in dspaths:
            timeseries[dspath.split('/')[-1]] = build_timeseries(timeseries_file[dspath][:], 
                                                                 sampling_interval=sampling_interval)

    return timeseries

def get_connectivity_matrix(timeseries, selection_method, similarity_measure):
    """
    Parameters
    ----------
    timeseries: dict or list of (nitime.timeseries.TimeSeries or numpy.ndarray)
        The N sets of timeseries of one subject.
        
    selection_method: str
        The name of the timeseries set transformation method.
        See `luigi.selection.TimeSeriesSelectorFactory.create_method` more information and the possible choices.
    
    similarity_method: str
        The name of the timeseries set transformation method.
        See `luigi.similarity_measure.SimilarityMeasureFactory.create_method` for more information and
        the possible choices.
    
    Returns
    -------
    connectivity: numpy.ndarray
        Matrix of shape (N, N)
    """
    selection  = TimeSeriesSelectorFactory.create_method(selection_method)
    similarity = SimilarityMeasureFactory. create_method(similarity_measure)

    # transform_timeseries(timeseries, selection_method, **kwargs)
    transformed_timeseries = transform_timeseries  (timeseries, selection)

    # calculate_connectivity(timeseries_set, measure, sampling_interval, lb=0, ub=None, **kwargs):
    return calculate_connectivity(transformed_timeseries, similarity, sampling_interval=TR)


def create_group_connectivites(timeseries_h5path, subj_groups, selection_method, similarity_measure):
    """
    Parameters
    ----------
    timeseries_h5path: str

    subj_groups: list of str

    selection_method: str
        The name of the timeseries set transformation method.
        See `luigi.selection.TimeSeriesSelectorFactory.create_method` more information and the possible choices.
    
    similarity_method: str
        The name of the timeseries set transformation method.
        See `luigi.similarity_measure.SimilarityMeasureFactory.create_method` for more information and
        the possible choices.
    
    Returns
    -------
    connectivities: dict
        Dictionary with subj_id -> connectivity_matrix
    """
    connectivities = OrderedDict()

    for subj_path in subj_groups:
        timeseries   = get_subject_timeseries  (timeseries_h5path, subj_path)
        connectivity = get_connectivity_matrix (timeseries, selection_method, similarity_measure)
        connectivities[subj_path.split('/')[-1]] = connectivity

    return connectivites

In [4]:
def plot_connectivity_matrix(x, title=None, show_ticklabels=False):
    sns.set(context="paper", font="monospace")
    #sns.set(style="darkgrid")
    #sns.set(rc={"figure.figsize": (6, 6)})
    fig, ax = plt.subplots()
    if title:
        plt.title(title)

    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    #sns.corrplot(x, annot=False, sig_stars=False, diag_names=False, cmap=cmap, ax=ax)
    sns.heatmap(x, linewidths=0, square=True, cmap=cmap)

    #plt.imshow(x, cmap='jet', interpolation='nearest')
    plt.setp(ax.get_yticklabels(), visible=show_ticklabels)
    plt.setp(ax.get_xticklabels(), visible=show_ticklabels)
    #if not show_ticklabels:
        #ax.set_xticklabels([])
        #ax.set_yticklabels([])

    fig.tight_layout()

#plot_connectivity_matrix(connectivity, 'test')

In [44]:
def get_connectivity_matrices(timeseries_h5path, selection_method, similarity_measure):
    """
    Parameters
    ----------
    selection_method: str
    
    similarity_measure: str

    Returns
    -------
    connectivities:

    """
    if aalconns_groupname in file_groups:
        connectivities = extract_datasets(timeseries_h5path, aalconns_groupname)
    else:
        print('Calculating connectivity matrices using {} and {}.'.format(selection_method, similarity_measure))
        connectivities = create_group_connectivites(timeseries_h5path, subj_groups, selection_method, similarity_measure)

        # save the connectivity matrices into the hdf file
        save_variables_to_hdf5(timeseries_h5path, 
                           {'{}-{}'.format(selection_method, similarity_measure): connectivities},
                           mode='a', 
                           h5path= aalconns_groupname)

    return connectivities

from functools      import partial
from boyle.parallel import parallel_function

selection_method   = 'eigen'
similarity_measure = 'mean_coherence'

def get_connectivity(timeseries_h5path, subj_path, selection_method, similarity_measure):
    timeseries = get_subject_timeseries(timeseries_h5path, subj_path)
    return get_connectivity_matrix(timeseries, selection_method, similarity_measure)


get_my_connectivities = partial(get_connectivity, timeseries_file=timeseries_h5path, 
                                selection_method=selection_method, similarity_measure=similarity_measure)

get_my_connectivities.parallel = parallel_function(get_my_connectivities, n_cpus=3)

start = time()
conns = get_my_connectivities.parallel(subj_groups)

connectivities = OrderedDict()
for subj_path, conn in zip(subj_groups, conns):
    connectivities[subj_path.split('/')[-1]] = conn

parallel_time = time() - start
print('parallel_time: {}'.format(parallel_time))


In [6]:
#for conn in connectivities:
#    plot_connectivity_matrix(connectivities[conn], conn)

## Classification

In [23]:
def extract_lower_triangular_matrix(x, k=0):
    """Return the lower triangular values of x without the main diagonal.
    
    Parameters
    ----------
    x: numpy.ndarray
        2D square matrix

    k : int, optional
        Diagonal above which to zero elements. 
        k = 0 (the default) is the main diagonal, 
        k < 0 is below it and 
        k > 0 is above.

    Returns
    -------
    features: numpy.ndarray
        vector
    """
    return x[np.tril_indices_from(x, k=k)]


def number_of_triangular_elements(x, k=0):
    """Return the number of elements that the lower triangular matrix of x has.
    
    Parameters
    ----------
    x: numpy.ndarray
        2D square matrix

    k : int, optional
        Diagonal above which to zero elements. 
        k = 0 (the default) is the main diagonal, 
        k < 0 is below it and 
        k > 0 is above.

    Returns
    -------
    n_elems: int
        number of elements in the triangular matrix
    """
    if not isinstance(x, np.ndarray):
        raise TypeError('Expected a numpy.ndarray, got a {}.'.format(type(x)))

    if x.ndim != 2:
        raise TypeError('Expected a 2D matrix, got a matrix with {} dimensions.'.format(x.ndim))

    if x.shape[0] != x.shape[1]:
        raise TypeError('Expected a square matrix, got a matrix with shape {}'.format(x.shape))

    if k == 0:
        rows    = x.shape[1]
        n_elems = 0.5 * ((rows + 1) * rows)
    else:
        ones    = np.ones_like(x)
        n_elems = np.sum(np.tril(ones, k=k)) 
    
    return int(n_elems)

In [45]:
from darwin.pipeline import ClassificationPipeline

# Diagonal above which to zero elements.
k = -1

# create the samples matrix
sample     = next (iter (connectivities.values()))
n_subjects = len(connectivities)
n_features = number_of_triangular_elements(sample, k=k)

feature_matrix = np.zeros((n_subjects, n_features), dtype=sample.dtype)

# get the data
selection_method   = 'eigen'
similarity_measure = 'mean_coherence'
connectivities = get_connectivity_matrices(timeseries_h5path, selection_method, similarity_measure)

# fill the feature matrix
for idx, conn in enumerate(connectivities):
    feature_matrix[idx, :] = extract_lower_triangular_matrix(connectivities[conn], k=k)

# -- test with darwin
# 'RandomForestClassifier', 'RBFSVC', 'LinearSVC', 'GMM', 
classifier_name = 'LinearSVC' #'linsvm'
cvmethod = 'loo'

pipe = ClassificationPipeline(clfmethod=classifier_name, cvmethod=cvmethod)

eprint('Classifying')
pipe.cross_validation(feature_matrix, np.array(subj_labels))
#results, metrics = pipe.cross_validation(feature_matrix, np.array(subj_labels))


(ClassificationResult(predictions=[0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0], probabilities=None, cv_targets=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0], best_parameters=OrderedDict([(0, {'C': 0.01}), (1, {'C': 0.01